### Teste

Verificar o tamanho da base quando tiramos a granaloridade de ibge e passamos para uf

In [9]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import zipfile as zf
import os
from io import BytesIO
import warnings
import mysql.connector
from mysql.connector import Error
import MySQLdb

warnings.filterwarnings('ignore')

In [10]:
from IPython import get_ipython
from nbformat import read
from IPython.core.interactiveshell import InteractiveShell

In [11]:
def dadosDB():
    '''
    Efetua a leitura do arquivo com as informações necessárias para acessar o banco de dados
    e criar o conector para acessar o banco de dados.
    '''
    global con
    with open('./pass/pass.txt') as file:
        lista = file.readlines()
        lista = list(x.replace('\n', '') for x in lista)
    con = mysql.connector.connect(host=lista[0], database=lista[1], user=lista[2], password=lista[3])
    return con

In [12]:
def conectar():
    '''
    Efetua a conexão com o banco de dados.
    Lê os dados necessário para conexão na função dadosDB().
    '''
    try:
        global con, cursor
        con = dadosDB()
        cursor = con.cursor()
        print('Conectado!')
    except Error as erro:
        print('Erro de conexão')

In [13]:
def finalizar():
    '''
    Irá finalizar a conexão após realizar a solicitação no DB.
    '''
    #dadosDB()
    if con.is_connected():
        cursor.close()
        con.close()
        print('Conexão encerrada!')

In [14]:
def AnoMes_Distinct():
    '''
    Query que irá trazer de forma distinta os meses que foram  importados (Campo AnoMes da tabela beneficiariosconsolidado)
    '''
    global AnoMes
    consulta_sql = "select distinct AnoMes from beneficiariosconsolidadoUF"
    dfa = pd.read_sql(consulta_sql, dadosDB())
    print('Consulta no banco de dados realizada!\n')
    AnoMes = list(str(x) for x in dfa['AnoMes']) #pegando a coluna AnoMes do df e passando para uma lista
    #finalizar()
    return AnoMes

In [15]:
def importar_base(base):
    '''
    Função que irá importar para a tabela beneficiariosconsolidado do banco de dados a base de beneficiarios que foi 
    carregar e tratada.
    '''
    try:
        conectar()
        cursor = con.cursor()
        print('Iniciando o processo de importação no Banco de Dados.\n')
        for x in base.index:
            sql = """
                INSERT INTO BeneficiariosConsolidadoUF (AnoMes, UF, Sexo, Faixa_Etaria,
                Cobertura_Assitencial_Plan, Tipo_Vinculo, Qtd_Benef_Ativos, Qtd_Benef_Aderidos, Qtd_Benef_Cancelados)
                values("%s", "%s", "%s", "%s", "%s", "%s", "%s", "%s", "%s");
                """ % (base['AnoMes'][x], base['UF'][x], base['Sexo'][x],
                       base['Faixa_Etaria'][x], base['Cobertura_Assitencial_Plan'][x], base['Tipo_Vinculo'][x], base['Qtd_Benef_Ativos'][x], 
                       base['Qtd_Benef_Aderidos'][x], base['Qtd_Benef_Cancelados'][x])
            cursor.execute(sql)
        con.commit()
        print('Atualização da base de dados realizada!')
    except MySQLdb.ProgrammingError as error :
        print("Database Failed!!")
        con.rollback()
    finally:
        finalizar() 

In [16]:
def select_base():
    global baseDB
    #planos = pd.DataFrame()
    consulta = "select * from BeneficiariosConsolidadoUF"
    baseDB = pd.read_sql(consulta, dadosDB())
    return baseDB

In [9]:
def temp(tag_meses, escolha):
    # Função temporaria que irá definir quantos meses serão executados para carregar no DB.
    # Devido o processo ser lento, preciso implentar essa função para particionar as cargas
    # Recebe as variaveis escolha (indica quantos meses será processado) e tag_meses(variável com os meses que tem para
    # para processar)
    tag_meses = tag_meses[0:escolha]
    return tag_meses

In [17]:
def web_scraping_meses():
    '''
    Função que irá acessar o site https://dadosabertos.ans.gov.br/FTP/PDA/informacoes_consolidadas_de_beneficiarios/,
    percorrer os links referente aos meses e salvar em uma variável 
    '''
    global tag_meses
    #escolha = int(input('Quantos meses serão processados: '))
    html = requests.get("https://dadosabertos.ans.gov.br/FTP/PDA/informacoes_consolidadas_de_beneficiarios/").content
    soup = BeautifulSoup(html, 'html.parser')
    tags = soup.find_all('a') # salvando as tags a
    print('Analizados os links na página web.\n')
    tag_meses = []
    AnoMes = AnoMes_Distinct()
    for tag in tags:
        tag_meses.append(tag.getText()) # salvando os textos que estão dentro das tags
    tag_meses = list(filter(lambda x: x[0:1].lower() in '20', tag_meses)) # separando apenas as strings que começa com '20'
    tag_meses = [x.rstrip('/') for x in tag_meses] # removendo a / no final da string
    tag_meses = [x for x in tag_meses if x not in AnoMes] # removendo o AnoMes que já está processado no banco
    #tag_meses = temp(tag_meses, escolha)
    print('Meses validados e separados para processamento.\n')
    # arquivos = link_meses(tag_meses)
    return tag_meses

In [11]:
def web_scraping_files(tag_meses, x):
    '''
    Função que irá pegar os valores da lista tag_meses que foi salvo no função web_scraping_meses e percorrer cada string
    para ir na página web referente aquele link e separar os links (nomes) dos arquivos que deverá realizar o download.
    '''
    global arquivos
    link = 'https://dadosabertos.ans.gov.br/FTP/PDA/informacoes_consolidadas_de_beneficiarios/'
    #for x in tag_meses:
    html_m = requests.get(f"{link}{x}/").content
    soup = BeautifulSoup(html_m, 'html.parser')
    tags_m = soup.find_all('a')
    tag_arquivos = []
    # print(soup.find_all('a'))
    for tag in tags_m:
        tag_arquivos.append(tag.getText()) # para abrir a tag
        #print(tag.getText()) # para abrir o texto da tag
    arquivos = list(filter(lambda x: x[0:4].lower() in 'ben2', tag_arquivos))
    print(f'Arquivos {arquivos} estão prontos para processamento.\n')
    return arquivos

In [12]:
def import_files(arquivos, x, y):
    '''
    Efetua o donwload do arquivo e carrega em um dataframe temporário.
    '''
    global temp_df, f
    #for y in arquivos:
    https = f'https://dadosabertos.ans.gov.br/FTP/PDA/informacoes_consolidadas_de_beneficiarios/{x}/{y}'
    file_zip = BytesIO(requests.get(https).content)
    with zf.ZipFile(file_zip) as z:
        temp = z.namelist()
        with z.open(f'{temp[0]}') as f:
            temp_df = pd.read_csv(f, sep=";", encoding = 'latin')
    return temp_df

In [13]:
def web_scraping():
    '''
    Função que unifica todos os processos de ETL para importação na tabela beneficiáriosconsolidado do DB.
    '''
    web_scraping_meses()
    for x in tag_meses:
        base = pd.DataFrame()
        web_scraping_files(tag_meses, x)        
        for y in arquivos:
            import_files(arquivos, x, y)
            group = temp_df.groupby([ '#ID_CMPT_MOVEL', 'CD_OPERADORA', 'SG_UF', 'TP_SEXO', 'DE_FAIXA_ETARIA', 
                                     'COBERTURA_ASSIST_PLAN', 'TIPO_VINCULO'])['QT_BENEFICIARIO_ATIVO', 'QT_BENEFICIARIO_ADERIDO',
                                     'QT_BENEFICIARIO_CANCELADO'].sum().reset_index()
            base = base.append(group, ignore_index=True)

            # print(f'O arquivo {y} tem {temp_df.shape[0]} registros, totalizando {df.shape[0]} registro na tabela final.\n')
        base.rename(
                columns= {'#ID_CMPT_MOVEL': 'AnoMes', 'CD_OPERADORA': 'CD_Operadora', 'SG_UF': 'UF', 'TP_SEXO': 'Sexo',
                          'DE_FAIXA_ETARIA': 'Faixa_Etaria',
                          'COBERTURA_ASSIST_PLAN': 'Cobertura_Assitencial_Plan','TIPO_VINCULO': 'Tipo_Vinculo',
                          'QT_BENEFICIARIO_ATIVO': 'Qtd_Benef_Ativos', 'QT_BENEFICIARIO_ADERIDO': 'Qtd_Benef_Aderidos',
                          'QT_BENEFICIARIO_CANCELADO': 'Qtd_Benef_Cancelados'}, inplace = True)
        base.to_csv(f"D:/Arquivos/{x}.csv")
        base = base.groupby(['AnoMes', 'UF', 'Sexo', 'Faixa_Etaria', 'Cobertura_Assitencial_Plan', 
                                'Tipo_Vinculo'])['Qtd_Benef_Ativos', 'Qtd_Benef_Aderidos',
                                     'Qtd_Benef_Cancelados'].sum().reset_index()
        importar_base(base)
    return base

In [20]:
base = web_scraping()

Analizados os links na página web.

Consulta no banco de dados realizada!

Meses validados e separados para processamento.

Arquivos ['ben202302_AC.zip', 'ben202302_AL.zip', 'ben202302_AM.zip', 'ben202302_AP.zip', 'ben202302_BA.zip', 'ben202302_CE.zip', 'ben202302_DF.zip', 'ben202302_ES.zip', 'ben202302_GO.zip', 'ben202302_MA.zip', 'ben202302_MG.zip', 'ben202302_MS.zip', 'ben202302_MT.zip', 'ben202302_PA.zip', 'ben202302_PB.zip', 'ben202302_PE.zip', 'ben202302_PI.zip', 'ben202302_PR.zip', 'ben202302_RJ.zip', 'ben202302_RN.zip', 'ben202302_RO.zip', 'ben202302_RR.zip', 'ben202302_RS.zip', 'ben202302_SC.zip', 'ben202302_SE.zip', 'ben202302_SP.zip', 'ben202302_TO.zip', 'ben202302_XX.zip'] estão prontos para processamento.

Conectado!
Iniciando o processo de importação no Banco de Dados.

Atualização da base de dados realizada!
Conexão encerrada!
Arquivos ['ben202303_AC.zip', 'ben202303_AL.zip', 'ben202303_AM.zip', 'ben202303_AP.zip', 'ben202303_BA.zip', 'ben202303_CE.zip', 'ben202303_DF.zi

In [72]:
base.head()

,AnoMes,UF,Sexo,Faixa_Etaria,Cobertura_Assitencial_Plan,Tipo_Vinculo,Qtd_Benef_Ativos,Qtd_Benef_Aderidos,Qtd_Benef_Cancelados
0,201406,AC,F,00 a 05 anos,Médico-hospitalar,DEPENDENTE,1266,41,11
1,201406,AC,F,00 a 05 anos,Médico-hospitalar,TITULAR,750,17,11
2,201406,AC,F,00 a 05 anos,Não identificado,DEPENDENTE,6,0,0
3,201406,AC,F,00 a 05 anos,Não identificado,TITULAR,1,0,0
4,201406,AC,F,00 a 05 anos,Odontológico,DEPENDENTE,157,8,3


In [17]:
teste = base.groupby([ '#ID_CMPT_MOVEL', 'CD_OPERADORA', 'SG_UF', 'TP_SEXO', 'DE_FAIXA_ETARIA', 'COBERTURA_ASSIST_PLAN', 'TIPO_VINCULO'])['QT_BENEFICIARIO_ATIVO', 'QT_BENEFICIARIO_ADERIDO',
       'QT_BENEFICIARIO_CANCELADO'].sum().reset_index()


In [59]:
ver = select_base()
ver

,AnoMes,UF,Sexo,Faixa_Etaria,Cobertura_Assitencial_Plan,Tipo_Vinculo,Qtd_Benef_Ativos,Qtd_Benef_Aderidos,Qtd_Benef_Cancelados
0,201407,AC,F,36 a 40 anos,Médico-hospitalar,TITULAR,1,0,0
1,201407,AC,F,00 a 05 anos,Médico-hospitalar,DEPENDENTE,1,0,0
2,201407,AC,F,21 a 25 anos,Médico-hospitalar,DEPENDENTE,1,0,0
3,201407,AC,M,31 a 35 anos,Médico-hospitalar,TITULAR,3,0,0
4,201407,AC,F,31 a 35 anos,Odontológico,DEPENDENTE,1,0,0
...,...,...,...,...,...,...,...,...,...
604285,201407,XX,F,56 a 60 anos,Médico-hospitalar,TITULAR,5,0,0
604286,201407,XX,F,61 ou mais,Médico-hospitalar,TITULAR,1,0,0
604287,201407,XX,M,51 a 55 anos,Médico-hospitalar,TITULAR,5,0,0
604288,201407,XX,M,56 a 60 anos,Médico-hospitalar,TITULAR,3,0,0


In [51]:
ver.groupby('AnoMes')['Qtd_Benef_Ativos'].sum()

AnoMes
201405    834113364
Name: Qtd_Benef_Ativos, dtype: int64

In [130]:
conectar()
cursor = con.cursor()
sql = "DROP TABLE IF EXISTS beneficiariosconsolidadouf"
cursor.execute(sql)

Conectado!


In [60]:
finalizar()

Conexão encerrada!


In [76]:
df.head()

NameError: name 'df' is not defined

In [18]:
teste = web_scraping_meses()

Analizados os links na página web.

Consulta no banco de dados realizada!

Meses validados e separados para processamento.



In [19]:
teste

['202302', '202303']